<a href="https://colab.research.google.com/github/dvschultz/stylegan2-training/blob/main/SG2_ADA_Generate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#StyleGAN2 Manipulation
#### Inference, Truncation, Interpolation, Mixing, and Projection

This notebook covers manipulating a StyleGAN2-ADA model to produce images and videos. It was made by [Derrick Schultz](https://twitter.com/dvsch) and [Lia Coleman](https://twitter.com/Lialialiacole) for their StyleGAN2 courses.

Parts of this notebook contain ideas and code from [Mikael Christensen](https://colab.research.google.com/drive/1ShgW6wohEFQtqs_znMna3dzrcVoABKIH) and [Dan Shiffman](https://www.youtube.com/watch?v=vEetoBuHj8g).

In [1]:
# mount Google Drive on the runtime
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [2]:
# create a symbolic link to a working directory
!ln -s /content/gdrive/My\ Drive/Colab\ Notebooks/monixypAI /mydrive

# navigate to the working directory
%cd /mydrive

/content/gdrive/My Drive/Colab Notebooks/monixypAI


In [3]:
%tensorflow_version 1.x
!nvidia-smi

TensorFlow 1.x selected.
Sat Apr  3 09:54:45 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    27W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                               

To make demoing easier, we can also add some functions to show images in Colab. Run the next cell to enable that code.

Unfortunately you can’t currently play video inside Colab, so you’ll have to download the videos to view them.

In [4]:
from IPython.display import Image, display

# Setup: Download the StyleGAN2 repo 

This will install all the necessary libraries to use the StyleGAN2 repo. Press the play button or `shift+return` to run each cell.

Only run the next cell once per session.


In [5]:
# !git clone https://github.com/dvschultz/stylegan2-ada
!pip install opensimplex

In [6]:
%cd stylegan2-ada

/content/gdrive/My Drive/Colab Notebooks/monixypAI/stylegan2-ada


### Download your trained model from Google Drive
Input the google id of your trained .pkl file. 
The -O option here is to specify the output file name and location.

In [ ]:
# !gdown --id 1YzZemZAp7BVW701_BZ7uabJWJJaS2g7v -O /content/network.pkl

#Generate Images

Also known as "Inference", "Evaluation" or "Testing" the model.


In [ ]:
# !python generate.py --help

##Options
`--network`

Make sure the `--network` argument points to your .pkl file. (My preferred method is to right click on the file in the Files pane to your left and choose `Copy Path`, then paste that into the argument after the `=` sign).

`--seeds`

This allows you to choose random seeds from the model. Remember that our input to StyleGAN is a 512-dimensional array. These seeds will generate those 512 values. Each seed will generate a different, random array. The same seed value will also always generate the same random array, so we can later use it for other purposes like interpolation.

In [51]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/generate'

!python generate.py generate-images \
                    --network=$network \
                    --outdir=$dir \
                    --seeds=0-1000

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 0 (0/1001) ...
Generating image for seed 1 (1/1001) ...
Generating image for seed 2 (2/1001) ...
Generating image for seed 3 (3/1001) ...
Generating image for seed 4 (4/1001) ...
Generating image for seed 5 (5/1001) ...
Generating image for seed 6 (6/1001) ...
Generating image for seed 7 (7/1001) ...
Generating image for seed 8 (8/1001) ...
Generating image for seed 9 (9/1001) ...
Generating image for seed 10 (10/1001) ...
Generating image for seed 11 (11/1001) ...
Generating image for seed 12 (12/1001) ...
Generating image for seed 13 (13/1001) ...
Generating image for seed 14 (14/1001) ...
Generating image for seed 15 (15/1001) ...
Generating image for seed 16 (16/1001) ...
Generatin

Let’s look at a handful of images. 

In [10]:
listOfImageNames = [f'{dir}/seed0000.png',
                    f'{dir}/seed0001.png',
                    f'{dir}/seed0002.png',
                    f'{dir}/seed0003.png',
                    f'{dir}/seed0004.png',
                    ]

for imageName in listOfImageNames:
    display(Image(filename=imageName, width=400))

Output hidden; open in https://colab.research.google.com to view.

Let’s zip the generated files and download them.

In [ ]:
# !zip -r generated-out.zip /content/stylegan2-ada/out

#### Generating Images With A Conditional model
A conditional model is a model trained with class labels. Specify the class you want output with the `--class` option.

In [13]:
# network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
# dir = './videos/generate_class'

# !python generate.py generate-images \
#                     --network=$network \
#                     --outdir=$dir \
#                     --class=1 \
#                     --seeds=0-100

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Traceback (most recent call last):
  File "generate.py", line 814, in <module>
    main()
  File "generate.py", line 809, in main
    func(**kwargs)
  File "generate.py", line 108, in generate_images
    label[:, class_idx] = 1
IndexError: index 1 is out of bounds for axis 1 with size 0


Note: The above command will output it to a directory called `out`. If you already have stuff in `out`, make sure to clear it out. To delete a entire folder in Colab:

In [ ]:
# !rm -r "./out/"

# Truncation Traversal


Truncation, well, truncates the latent space. This can have a subtle or dramatic affect on your images depending on the value you use. Most people choose between 0.5 and 1.0, but technically it's infinite. 

Below you can take one seed and look at the changes to it across any truncation amount. -1 to 1 will be pretty realistic images, but the further out you get the weirder it gets.

###Options 
`--network`: Again, point this to your .pkl file.

`--seed`: Pass this only one seed. Pick a favorite from your generated images.

`--start`: Starting truncation value.

`--stop`: Stopping truncation value. This should be larger than the start value. (Will probably break if its not).

`--increment`: How much each frame should increment the truncation value. Make this really small if you want a long, slow interpolation. (stop-start/increment=total frames)

In [18]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/truncation-traversal'

# truncation-traversal-seed1-start-2.0-stop2.0.mp4
# !python generate.py truncation-traversal \
#                     --network=$network \
#                     --seed=1 \
#                     --start=-2.0 \
#                     --stop=2.0 \
#                     --increment=0.25 \
#                     --outdir=$dir \
#                     --fps=30

# 2_truncation-traversal-seed1-start-2.0-stop2.0.mp4
# !python generate.py truncation-traversal \
#                     --network=$network \
#                     --seed=1 \
#                     --start=-2.0 \
#                     --stop=2.0 \
#                     --increment=0.025 \
#                     --outdir=$dir \
#                     --fps=30

# 3_truncation-traversal-seed666-start-10.0-stop10.0.mp4
# !python generate.py truncation-traversal \
#                     --network=$network \
#                     --seed=666 \
#                     --start=-10.0 \
#                     --stop=10.0 \
#                     --increment=0.25 \
#                     --outdir=$dir \
#                     --fps=30

# 4_truncation-traversal-seed123456-start-1.0-stop1.0.mp4
# !python generate.py truncation-traversal \
#                     --network=$network \
#                     --seed=123456 \
#                     --start=-1.0 \
#                     --stop=1.0 \
#                     --increment=0.01 \
#                     --outdir=$dir \
#                     --fps=30

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating truncation -1.00
Generating truncation -0.99
Generating truncation -0.98
Generating truncation -0.97
Generating truncation -0.96
Generating truncation -0.95
Generating truncation -0.94
Generating truncation -0.93
Generating truncation -0.92
Generating truncation -0.91
Generating truncation -0.90
Generating truncation -0.89
Generating truncation -0.88
Generating truncation -0.87
Generating truncation -0.86
Generating truncation -0.85
Generating truncation -0.84
Generating truncation -0.83
Generating truncation -0.82
Generating truncation -0.81
Generating truncation -0.79
Generating truncation -0.78
Generating truncation -0.77
Generating truncation -0.76
Generating truncation -0.75
Generating truncatio

There should now be a video called something like: `truncation-traversal-seed1-start-2.0-stop2.0.mp4` in the `tt` folder. You can download just the video by right clicking on it. If you want all the individual frames that are there too, zip the `tt` folder and download it.

In [ ]:
# !zip -r generated-tt.zip /content/stylegan2-ada/tt

# Interpolations

Interpolation is the process of generating very small changes to a vector in order to make it appear animated from frame to frame.

We’ll look at two different examples of interpolation: a linear interpolation and a random noise loop.

Both methods require the following options:

`--network`: path to your .pkl file

`--walk-type`: Walk type defines the type of interpolation you want. In some cases it can also specify whether you want the z space or the w space.

`--frames`: How many frames you want to produce. Use this to manage the length of your video.

`--trunc`: truncation value


### Linear Interpolation

Linear interpolation generates a linear path from one seed to another. The makers of StyleGAN say that doing this in the w space produces the best disentangled interpolations. But let’s start by looking at it in z space.

`--seeds`: Use images you generated to control the interpolation points. If your first and last seed are the same this will produce a loop (nice for Instagram and gifs!)

In [20]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/generate-latent-walk/z-walk'

# 1_walk-z-line0-1-0-24fps.mp4
# !python generate.py generate-latent-walk \
#                     --network=$network \
#                     --walk-type="line-z" \
#                     --seeds=0,1,0 \
#                     --outdir=$dir \
#                     --frames=720

# 2_walk-z-line2-38-2-24fps.mp4
!python generate.py generate-latent-walk \
                    --network=$network \
                    --walk-type="line-z" \
                    --seeds=2,38,2 \
                    --outdir=$dir \
                    --frames=360

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/362 ...
Generating image for step 1/362 ...
Generating image for step 2/362 ...
Generating image for step 3/362 ...
Generating image for step 4/362 ...
Generating image for step 5/362 ...
Generating image for step 6/362 ...
Generating image for step 7/362 ...
Generating image for step 8/362 ...
Generating image for step 9/362 ...
Generating image for step 10/362 ...
Generating image for step 11/362 ...
Generating image for step 12/362 ...
Generating image for step 13/362 ...
Generating image for step 14/362 ...
Generating image for step 15/362 ...
Generating image for step 16/362 ...
Generating image for step 17/362 ...
Generating image for step 18/362 ...
Generating image for step 1


Next let’s look at linear interpolation in w space. To do this we set `--walk-type` to `line-w`.

I recommend using the exact same seeds so you see the difference. It’s often very subtle but it is different.


In [22]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/generate-latent-walk/w-walk'

# 5_walk-w-line45-92-24-45-24fps.mp4
!python generate.py generate-latent-walk \
                    --network=$network \
                    --walk-type="line-w" \
                    --seeds=45,92,24,45 \
                    --outdir=$dir \
                    --frames=720

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/723 ...
Generating image for step 1/723 ...
Generating image for step 2/723 ...
Generating image for step 3/723 ...
Generating image for step 4/723 ...
Generating image for step 5/723 ...
Generating image for step 6/723 ...
Generating image for step 7/723 ...
Generating image for step 8/723 ...
Generating image for step 9/723 ...
Generating image for step 10/723 ...
Generating image for step 11/723 ...
Generating image for step 12/723 ...
Generating image for step 13/723 ...
Generating image for step 14/723 ...
Generating image for step 15/723 ...
Generating image for step 16/723 ...
Generating image for step 17/723 ...
Generating image for step 18/723 ...
Generating image for step 1

#### Spherical Linear Interpolation (slerp)

This gets a little heady, but technically linear interpolations are not the best in high-dimensional GANs. [This github link](https://github.com/soumith/dcgan.torch/issues/14) is one of the more popular explanations ad discussions.

In reality I do not find a huge difference between linear and spherical interpolations (the difference in z- and w-space is enough in many cases), but I’ve implemented slerp here for anyone interested.

In [23]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/generate-latent-walk/z-sphere'

# 6_walk-z-sphere28-47-52-28-24fps.mp4
!python generate.py generate-latent-walk \
                    --network=$network \
                    --walk-type="sphere-z" \
                    --seeds=28,47,52,28 \
                    --outdir=$dir \
                    --frames=720

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
slerp
Generating image for step 0/723 ...
Generating image for step 1/723 ...
Generating image for step 2/723 ...
Generating image for step 3/723 ...
Generating image for step 4/723 ...
Generating image for step 5/723 ...
Generating image for step 6/723 ...
Generating image for step 7/723 ...
Generating image for step 8/723 ...
Generating image for step 9/723 ...
Generating image for step 10/723 ...
Generating image for step 11/723 ...
Generating image for step 12/723 ...
Generating image for step 13/723 ...
Generating image for step 14/723 ...
Generating image for step 15/723 ...
Generating image for step 16/723 ...
Generating image for step 17/723 ...
Generating image for step 18/723 ...
Generating image for 

In [24]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/generate-latent-walk/w-sphere'

# 7_walk-w-sphere60-97-60-24fps.mp4
!python generate.py generate-latent-walk \
                    --network=$network \
                    --walk-type="sphere-w" \
                    --seeds=60,97,60 \
                    --outdir=$dir \
                    --frames=720

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
slerp
Generating image for step 0/722 ...
Generating image for step 1/722 ...
Generating image for step 2/722 ...
Generating image for step 3/722 ...
Generating image for step 4/722 ...
Generating image for step 5/722 ...
Generating image for step 6/722 ...
Generating image for step 7/722 ...
Generating image for step 8/722 ...
Generating image for step 9/722 ...
Generating image for step 10/722 ...
Generating image for step 11/722 ...
Generating image for step 12/722 ...
Generating image for step 13/722 ...
Generating image for step 14/722 ...
Generating image for step 15/722 ...
Generating image for step 16/722 ...
Generating image for step 17/722 ...
Generating image for step 18/722 ...
Generating image for 

### Noise Loop Interpolation

If you want to just make a random but fun interpolation of your model the noise loop is the way to go. It creates a random path thru the z space to show you a diverse set of images.

`--diameter`: This controls how "wide" the loop is. Make it smaller to show a less diverse range of samples. Make it larger to cover a lot of samples. This plus `--frames` can help determine how fast the video feels.

`--start_seed`: this allows you to change your starting place in the z space. Note: this value has nothing to do with the seeds you use to generate images. It just allows you to randomize your start point (and if you want to return to it you can use the same seed multiple times).

In [25]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/generate-latent-walk/noise'

# 8_walk-z-noiseloop-seed7-24fps.mp4
!python generate.py generate-latent-walk \
                    --network=$network \
                    --walk-type="noiseloop" \
                    --start_seed=7 \
                    --outdir=$dir

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/240 ...
Generating image for step 1/240 ...
Generating image for step 2/240 ...
Generating image for step 3/240 ...
Generating image for step 4/240 ...
Generating image for step 5/240 ...
Generating image for step 6/240 ...
Generating image for step 7/240 ...
Generating image for step 8/240 ...
Generating image for step 9/240 ...
Generating image for step 10/240 ...
Generating image for step 11/240 ...
Generating image for step 12/240 ...
Generating image for step 13/240 ...
Generating image for step 14/240 ...
Generating image for step 15/240 ...
Generating image for step 16/240 ...
Generating image for step 17/240 ...
Generating image for step 18/240 ...
Generating image for step 1

### Circular Loop Interpolation

The noise loop is, well, noisy. This circular loop will feel much more even, while still providing a random loop.

I recommend using a higher `--diameter` value than you do with noise loops. Something between `50.0` and `500.0` alongside `--frames` can help control speed and diversity.

In [26]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/generate-latent-walk/circular'

# 9_walk-z-circularloop-seed51-24fps.mp4
!python generate.py generate-latent-walk \
                    --network=$network \
                    --walk-type="circularloop" \
                    --start_seed=51 \
                    --diameter=300.0 \
                    --frames=720 \
                    --outdir=$dir

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for step 0/721 ...
Generating image for step 1/721 ...
Generating image for step 2/721 ...
Generating image for step 3/721 ...
Generating image for step 4/721 ...
Generating image for step 5/721 ...
Generating image for step 6/721 ...
Generating image for step 7/721 ...
Generating image for step 8/721 ...
Generating image for step 9/721 ...
Generating image for step 10/721 ...
Generating image for step 11/721 ...
Generating image for step 12/721 ...
Generating image for step 13/721 ...
Generating image for step 14/721 ...
Generating image for step 15/721 ...
Generating image for step 16/721 ...
Generating image for step 17/721 ...
Generating image for step 18/721 ...
Generating image for step 1

# Near Neighbors

**Added by popular demand**

Let’s say you have a seed you like, but want to see other images like it to see if there’s something better. Now you can with the `near-neighbor` argument.

### Options
`--network`, `--seeds`, and `--truncation_psi` work the same as above.

`--diameter`: this sets how far away from the seed you want to generate images. `.0000001` is really close, `.5` is reallly far.

`--num_samples`: how many samples you want to produce

`--save_vector`: this will save the vector as a file in the .npy format. You can then use this for interpolation (not super well supported right now, but can be used manually—see the Projection code for an example of reading a .npy file and interpolating it).




In [31]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/neighbors'

!python generate.py generate-neighbors \
                    --network=$network \
                    --diameter=0.5 \
                    --num_samples=50 \
                    --seeds=2 \
                    --outdir=$dir

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating image for seed 2 (1/1) ...


In [ ]:
# !zip -r neighbors.zip /content/stylegan2-ada/neighbors

# Style Mixing

Since seeds are just points in a vector space, we can do math things to them, like adding them together. You could do this thru linear interpolation and using the middle frame, but if you want to visualize a number of options here’s a simple script to do it. This takes a number of seeds to produce a grid showing what happens when you add the row and column together (this will make more sense after running it).

In [32]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/style-mix'

!python style_mixing.py \
        --network=$network \
        --outdir=$dir \
        --rows=85,100,75,458,1500 \
        --cols=55,821,1789,293

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Generating W vectors...
Generating images...
Generating style-mixed images...
Saving images...
Saving image grid...


# Flesh Digressions

[Flesh Digressions](https://aydao.ai/artwork/2020/01/17/fleshdigressions.html) is a technique from [aydao](https://twitter.com/aydaogman). This command will output a flesh digressions video called `circular-25-10-2020-09-00-34-PM.mp4`.

In [ ]:
!python aydao_flesh_digressions.py --pkl "/content/network.pkl" --psi=0.6 --radius_small=10.0 --radius_large=800.00

# Projection

Projection is the process of taking an image from outside the model and finding its nearest representation inside the model. 

###Options 
`--target`: upload an image that you want to project into the model, and then fill the `--target` option with the path to it. This can be any image, as long as it has the same aspect ratio as the dataset that the network was trained on.

`--network`: your model. Just for this example, we're using the FFHQ network in the `--network` option because it's really easy to see what projection is doing when its projection on faces. But of course, you can input your own model as well!

In [50]:
network = '/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl'
dir = './videos/projection'
target= './videos/veins.jpg'

!python projector.py \
        --network=$network \
        --outdir=$dir \
        --target=$target

Loading networks from "/mydrive/stylegan2-ada/results/00026-monixyp_tfrecords-mirror-mirrory-11gb-gpu-bg-resumecustom/network-snapshot-000068.pkl"...
Setting up TensorFlow plugin "fused_bias_act.cu": Loading... Done.
Setting up TensorFlow plugin "upfirdn_2d.cu": Loading... Done.
Projector: Computing W midpoint and stddev using 10000 samples...
Projector: std = 8.23895
Projector: Setting up noise inputs...
Projector: Building image output graph...
Projector: Building loss graph...
Projector: Building noise regularization graph...
Projector: Setting up optimizer...
Projector: Preparing target images...
Projector: Initializing optimization state...
100% 1000/1000 [04:07<00:00,  4.04it/s, dist=0.4776, loss=0.48]
